In [1]:
from datasets import load_dataset

dataset = load_dataset("microsoft/ms_marco","v1.1")

Generating validation split:   0%|          | 0/10047 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/82326 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9650 [00:00<?, ? examples/s]

In [2]:
len(dataset['train'])

82326

In [3]:
len(dataset['test'])

9650

In [9]:
from tqdm import tqdm 
total_passages = set()
for r in dataset['test']:
    for t in r['passages']['passage_text']:
        total_passages.add(t)
for r in dataset['train']:
    for t in r['passages']['passage_text']:
        total_passages.add(t)
for r in dataset['validation']:
    for t in r['passages']['passage_text']:
        total_passages.add(t)

In [10]:
print(len(total_passages))

767675


In [20]:
d = dataset['validation']
i = 200
negatives = [t for idx, t in enumerate(d[i]['passages']['passage_text']) if d[i]['passages']['is_selected'][idx]==0]
positives = [t for idx, t in enumerate(d[i]['passages']['passage_text']) if d[i]['passages']['is_selected'][idx]==1]
print("Question:",d[i]['query'])
print()
print("True Passage")
for p in positives:
    print(p)
print()
print("False Passage")
for idx, n in enumerate(negatives):
    print(n)

Question: who wrote Nothing compares to you

True Passage
Nothing Compares 2 U  is a song originally written and composed by Prince for one of his side projects, The Family. It was later made famous by Irish recording artist Sinead Sinéad'O, connor whose arrangement was released as the second single from her second studio, Album I Do Not Want What I'haven T. got Speaking about her relationship with Prince in an interview with Norwegian station NRK in November 2014 Sinead sinéad, Said i did meet him a couple of. Times we'didn t get on at. All in, fact we had a-punch.. Up she: Continued he summoned me to his house 'After Nothing compares 2.' U i made it without. him

False Passage
Answer by Mecoach50. Confidence votes 22. Nothing Compares 2 U was written by Prince and recorded by Sinead O'Connor. Yes it was written by Prince, but no it was not first recorded by Sinead O' Connor. The first to record the song was a Prince created band called The Family. The song It Had To Be You was origin

In [2]:
from datasets import load_dataset
qap = load_dataset('enelpol/rag-mini-bioasq', 'question-answer-passages')
corpus = load_dataset('enelpol/rag-mini-bioasq', 'text-corpus')

Generating train split:   0%|          | 0/4012 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/707 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/40181 [00:00<?, ? examples/s]

In [9]:
from datasets import concatenate_datasets
q2 = qap['train'].rename_column('relevant_passage_ids','gold_titles')
q3 = qap['test'].rename_column('relevant_passage_ids','gold_titles')
q= concatenate_datasets([q2,q3])
q.to_json('data/mini-bioasq_query.jsonl',orient='records',lines=True)

Creating json from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

2102860

In [7]:
c2 = corpus['test'].rename_column('id','title')
c2.to_json('data/mini-bioasq_t2p.jsonl', orient="records", lines=True)

Creating json from Arrow format:   0%|          | 0/41 [00:00<?, ?ba/s]

61976973